In [2]:
import pandas as pd
import numpy as np
from pmdarima.arima import auto_arima
from pandas.tseries.offsets import DateOffset
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Função para preparar uma visão geral de todas as diárias de UTI por mês/ano
def prepare_overall_time_series(data):
    # Remover a coluna do estabelecimento e o total, pois não são necessários para a análise agregada
    data_overall = data.drop(['Estab.Saúde-Cidade', 'Total'], axis=1)
    
    # Converter as datas para o formato datetime e garantir que são o índice
    data_overall.columns = pd.to_datetime(data_overall.columns, errors='coerce')
    data_overall = data_overall.transpose()
    
    # Limpar os dados, substituir '-' por NaN e converter para float
    data_overall.replace('-', np.nan, inplace=True)
    data_overall = data_overall.apply(pd.to_numeric, errors='coerce')
    
    # Preencher valores ausentes com a média da coluna
    data_overall.fillna(data_overall.mean(), inplace=True)
    
    # Sumarizar os dados para obter o total por mês
    data_overall_sum = data_overall.sum(axis=1)
    
    # Definir uma frequência mensal para o índice
    data_overall_sum.index = pd.date_range(start=data_overall_sum.index[0], periods=len(data_overall_sum), freq='M')
    
    return data_overall_sum

# Função de previsão usando SARIMAX com auto_arima

def forecast_time_series(data, steps=12):     # steps : determina a quantidade de meses previstos
    
    # Use auto_arima para encontrar os melhores hiperparâmetros do modelo SARIMA
    model = auto_arima(data, seasonal=True, m=12, stepwise=True, trace=True, error_action='ignore',
                      suppress_warnings=True, max_order=None)
    
    # Ajustar o modelo SARIMAX com os hiperparâmetros encontrados
    result = SARIMAX(data, order=model.order, seasonal_order=model.seasonal_order)
    fitted_model = result.fit(disp=False)
    
    # Fazer previsões
    forecast = fitted_model.get_forecast(steps=steps)
    forecast_index = pd.date_range(data.index[-1] + DateOffset(months=1), periods=steps, freq='M')
    forecast_values = forecast.predicted_mean
    forecast_conf_int = forecast.conf_int()
    
    # Construir dataframe de previsão
    forecast_df = pd.DataFrame({
        'Forecast': forecast_values,
        'Lower CI': forecast_conf_int.iloc[:, 0],
        'Upper CI': forecast_conf_int.iloc[:, 1]
    }, index=forecast_index)
    
    return forecast_df

# Carregar o arquivo Excel
file_path = r'C:\Users\DuduCuco\Documents\TCC-Git\DoencasRespiratorias\BancoDeDados\DiasPermHosp.xlsx'
data = pd.read_excel(file_path)

# Preparar a série temporal agregada
overall_time_series = prepare_overall_time_series(data)

# Usando a função de previsão que foi definida anteriormente
overall_forecast = forecast_time_series(overall_time_series)

# Exibir as previsões
print(overall_forecast)

KeyError: "['Estab.Saúde-Cidade', 'Total'] not found in axis"